In [ ]:
import torch
import torchvision
import torchvision.transforms as tr # 데이터를 불러 오면서 바로 전처리를 할 수 있게 해주는 library
from torch.utils.data import DataLoader , Dataset
import numpy as np

### 1. 파이토치 제공 데이터 사용

In [ ]:
# 필요한 전처리 작업을 Compose안에 나열 - 데이터를 불러올 때, 미리 전처리를 할 수 있다.
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])
# Transforms on PIL Image
# Pad, Grayscale, RandomCrop, Normalize, ...

In [ ]:
# transform : 들어오는 데이터에 대해 전처리를 해주겠다
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainset[0][0].size() # 채널이 앞에 있다. 그리고 높이 너비

torch.Size([3, 8, 8])

In [ ]:
# 데이터 로더를 통해 배치사이즈와 셔플을 결정할 수 있다.
# num_workers는 데이터를 로드할 때 subprocess를 몇 개 쓸 것인지
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [ ]:
len(trainloader) # CIFAR dataset의 train이미지 개수가 5만개 -> 배치사이즈를 50으로 했기에 1000개의 set이 된 것

1000

In [ ]:
# 실제값을 보고 싶을 땐, iter와 next함수 - 하나씩 불러오겠다.
dataiter = iter(trainloader)
images, labels = next(dataiter)

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a74801191b0>
    <function _MultiProcessingDataLoaderIter.__del__ at 0x7a74801191b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
self._shutdown_workers()  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
self._shutdown_workers()    
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
if w.is_alive():    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive

    assert self._parent_pid == os.getpid(), 'can only test a child process'
  File "/usr/lib/python3.10/multiprocessing/process.py", line 1

In [ ]:
images.size() # 배치사이즈, 채널, 높이, 너비

torch.Size([50, 3, 8, 8])

### 2. 같은 클래스 별 폴더 이미지 데이터 이용

In [ ]:
# ./class/tiger
# ./class/lion 으로 잘 나누었다면
# torchvision.datasets.ImageFolder 이용
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
# 클래스 내를 구분해서 폴더를 만들어 두었기에, 레이블을 자동으로 매겨줌
trainset = torchvision.datasets.ImageFolder(root='/content/class', transform=transf)
trainloader = DataLoader(trainset, batch_size=1, shuffle=False, num_workers=2)
print(len(trainloader))
trainset[0][0].size() # 3x16x16

FileNotFoundError: ignored

### 3. 개인 데이터 사용(2 types)

In [6]:
# import preprocessing

train_images = np.random.randint(256, size=(20, 32, 32, 3))
train_labels = np.random.randint(2, size=(20, 1))

# preprocessing...
# train_images, train_labels = preprocessing(train_images, train_labels)

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [ ]:
# 아까 불러왔던, from torch.utils.data import DataLoader, Dataset에서
# Dataset의 클래스를 상속 받을 클래스를 만들어 준다. 이름은 마음대로
class TensorData(Dataset):
  # 데이터가 외부에 있기 때문에 받을 수 있도록, x_data, y_data
  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.x_data = self.x_data.permute(0, 3, 1, 2) # 이미지 개수, 채널 수 , 이미지 너비, 높이 순서로 변경
    self.y_data = torch.LongTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [ ]:
train_data = TensorData(train_images, train_labels) # 인스턴스 생성
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [ ]:
train_data[0][0].size() # 순서가 잘 바뀐 것을 확인할 수 있다.

torch.Size([3, 32, 32])

In [ ]:
print(train_data[0][0]) # 3, 32, 32
print(train_data[0][1]) # label
print(train_data[1][0]) # 3, 32, 32
print(train_data[1][1]) # label

tensor([[[ 71., 183.,  82.,  ..., 229.,  35., 169.],
         [126.,  44.,  77.,  ..., 191., 141., 153.],
         [ 64., 120.,  41.,  ...,  69.,   3., 149.],
         ...,
         [110.,  58., 221.,  ..., 113.,  13., 135.],
         [172., 136., 214.,  ..., 187., 204., 102.],
         [246.,   2., 251.,  ...,  61., 214., 158.]],

        [[ 65., 239., 219.,  ..., 189.,  45., 254.],
         [121.,  81., 135.,  ..., 115., 204.,  44.],
         [ 75., 134., 243.,  ...,  85., 230.,  40.],
         ...,
         [191., 188., 226.,  ...,   7., 218.,  42.],
         [193., 178.,  75.,  ..., 118.,  33., 189.],
         [ 44.,  66., 111.,  ...,  45., 198., 196.]],

        [[ 71., 254.,  94.,  ...,  12.,   7.,  56.],
         [ 56.,  56.,  61.,  ..., 230., 140., 206.],
         [254., 160., 118.,  ..., 232., 137.,  67.],
         ...,
         [ 96., 103., 125.,  ..., 170., 162., 140.],
         [144., 254., 212.,  ...,  97., 147., 145.],
         [119., 132., 233.,  ...,  82., 212.,  55.]]]

In [ ]:
# trainloader로 배치사이즈로 나눈 것도 확인
dataiter = iter(train_loader)
images, labels = next(dataiter)

In [ ]:
images.size()

torch.Size([10, 3, 32, 32])

### 2.에서 개인적으로 torchvision.datasets.ImageFolder를 쓰지 않는 이유!

* 폴더 정리를 못 하는 경우에는 사용불가하다
  - 다른 작업과 공용으로 사용하는 경우
  - 폴더가 아닌 SQL 같은 곳에서 넘어오는 경우

* 파이토치에서 제공하는 transform의 종류가 제한적이다
  - 디테일한 부분은 따로 함수를 짜야 한다.

### 따라서 3번과 같이 데이터 작업을 사용한다.


# + 파이토치로 무조건 전처리 하기 (고급)

그런데 개인 데이터에다가 transform까지 적용!
사실 조금 까다롭다.

우리가 가지고 있는 데이터를 transform하기 위해서는 아래와 같은 양식을 외우자!

In [2]:
import torch
import torchvision
import torchvision.transforms as tr # 데이터를 불러 오면서 바로 전처리를 할 수 있게 해주는 library
from torch.utils.data import DataLoader , Dataset
import numpy as np

In [11]:
from torch.utils.data import Dataset
# 위와 다른 점은 미리 작업을 하고 데이터를 받는 것이 아닌,
# 여기서 transform을 받는다. 그리고 이 안에서 전처리 작업을 하게 되는 것이다.
class MyDatasets(Dataset):

  def __init__(self, x_data, y_data, transform=None):

    self.x_data = x_data
    self.y_data = y_data
    self.transform = transform
    self.len = len(y_data)

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform:
      sample = self.transform(sample)

    return sample

  def __len__(self):
    return self.len

# 따로 텐서로 변환하는 클래스를 만든다.
class ToTensor:
  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2, 0, 1)
    return inputs, torch.LongTensor(labels)

class LinearTensor:

  def __init__(self, slope=1, bias=0):
    self.slope = slope
    self.bias = bias

  def __call__(self, sample):
    inputs, labels = sample
    inputs = self.slope*inputs + self.bias
    return inputs, labels


In [12]:
trans = tr.Compose([ToTensor(), LinearTensor(2, 5)]) # tr.ToTensor가 아닌 위의 클래스 ToTensor
# tr에서 제공하는 것이 아닌 굳이 클래스를 만들어서 사용하는 이유는
# 들어오는 데이터가 이미지는 PIL 이미지 형태여야 한다. 그래야 사용가능
# but 우리가 쓸 데이터 train_images는 넘파이 -> 에러가 난다
# 따라서 따로 클래스를 만들어 trans를 통해 MyDataset클래스로 보내는 것
# 그래도 너무 쓰고 싶다!! 맨 아래 설명
#
ds1 = MyDatasets(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)
# 배치를 나누는 이유는 학습할 떄 SGD인 optimizer를 사용하는데, 학습 때 전체 데이터를 사용하면 너무 느리기에
# 배치를 나눠 일부분의 데이터만 넣어 학습하게 된다.

In [13]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [15]:
dataiter1 = iter(train_loader1)
images1, labels1 = next(dataiter1)

In [16]:
images1 # 원래는 255까지 인데, slope을 곱하고, bias를 더함으로 값이 커진 것을 보면, 잘 계산이 된 것이다.

tensor([[[[209.,  69., 105.,  ..., 195., 413., 241.],
          [ 71., 381., 421.,  ..., 207., 203., 461.],
          [503., 459., 233.,  ..., 225., 315.,  35.],
          ...,
          [361., 253.,  99.,  ..., 455., 235., 339.],
          [199., 187., 343.,  ..., 283., 439., 375.],
          [ 91., 139., 413.,  ..., 395.,  67., 499.]],

         [[175., 159.,  27.,  ..., 311., 275., 363.],
          [ 97.,  89.,  47.,  ..., 399.,  69., 181.],
          [473., 423., 451.,  ..., 103., 401., 105.],
          ...,
          [275.,  33., 487.,  ..., 459., 121., 219.],
          [495.,  71., 427.,  ..., 151., 137., 365.],
          [201., 163.,  65.,  ..., 195., 207., 435.]],

         [[267., 299., 427.,  ..., 445., 367., 309.],
          [123., 235.,  39.,  ...,  41., 493., 231.],
          [217., 195., 185.,  ..., 495., 187., 235.],
          ...,
          [371., 247., 211.,  ..., 219., 365., 147.],
          [ 73., 243., 509.,  ..., 203., 491., 139.],
          [261., 407., 495.,  ...

In [17]:
images1.size()

torch.Size([10, 3, 32, 32])

trans = tr.Compose([ToTensor(), LinearTensor(2, 5)])
* tr.ToTensor가 아닌 위의 클래스 ToTensor
* tr에서 제공하는 것이 아닌 굳이 클래스를 만들어서 사용하는 이유는
* 들어오는 데이터가 이미지는 PIL 이미지 형태여야 한다. 그래야 사용가능
* but 우리가 쓸 데이터 train_images는 넘파이 -> 에러가 난다
* 따라서 따로 클래스를 만들어 텐서로 바꾸고 해주는 것
* 그래도 너무 쓰고 싶다!! 아래부터 설명

In [19]:
class MyDataset(Dataset):

  def __init__(self, x_data, y_data, transform=None):
    self.x_data =x_data
    self.y_data = y_data
    self.transform = transform
    self.len = len(y_data)

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform:
      sample = self.transform(sample)

  def __len__(self):
    return self.len

# transform을 사용할 클래스
class MyTransform:

  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2, 0, 1)
    labels = torch.FloatTensor(labels)

    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128) ,tr.ToTensor(), tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    final_output = transf(inputs)

    return final_output, labels

In [22]:
ds2 = MyDatasets(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True)
# 배치를 나누는 이유는 학습할 떄 SGD인 optimizer를 사용하는데, 학습 때 전체 데이터를 사용하면 너무 느리기에
# 배치를 나눠 일부분의 데이터만 넣어 학습하게 된다.

In [23]:
first_data = ds2[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [25]:
dataiter2 = iter(train_loader2)
images2, labels2 = next(dataiter2)

In [27]:
images2.size() # 128로 resize

torch.Size([10, 3, 128, 128])

In [28]:
# normalize 완료
images2

tensor([[[[-0.4510, -0.4510, -0.4588,  ..., -0.2078, -0.1843, -0.1843],
          [-0.4510, -0.4510, -0.4588,  ..., -0.2078, -0.1843, -0.1843],
          [-0.4431, -0.4431, -0.4431,  ..., -0.2000, -0.1922, -0.1922],
          ...,
          [-0.5843, -0.5843, -0.5686,  ..., -0.1529, -0.0745, -0.0745],
          [-0.7333, -0.7333, -0.7098,  ..., -0.2392, -0.1765, -0.1765],
          [-0.7333, -0.7333, -0.7098,  ..., -0.2392, -0.1765, -0.1765]],

         [[ 0.9373,  0.9373,  0.9294,  ...,  0.5451,  0.5451,  0.5451],
          [ 0.9373,  0.9373,  0.9294,  ...,  0.5451,  0.5451,  0.5451],
          [ 0.8275,  0.8275,  0.8118,  ...,  0.4275,  0.4118,  0.4118],
          ...,
          [ 0.3725,  0.3725,  0.3882,  ...,  0.3412,  0.4118,  0.4118],
          [ 0.4588,  0.4588,  0.4588,  ...,  0.4588,  0.5686,  0.5686],
          [ 0.4588,  0.4588,  0.4588,  ...,  0.4588,  0.5686,  0.5686]],

         [[ 0.8039,  0.8039,  0.6314,  ...,  0.2941,  0.3569,  0.3569],
          [ 0.8039,  0.8039,  